In [3]:
# imports packages
import argparse, sys, json, time
import pandas as pd
import numpy as np
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException


# load functions
import import_ipynb
import functions as func


# load credentials
client = GoogleAdsClient.load_from_storage(version="v13")

# set customer id (google ads)
customer_id = '1374815311'


# set seed keyword
keywords = ['test']


# set number of iterations
iterations = 1


# set name of topic
name = 'test'


# set location
# https://developers.google.com/google-ads/api/reference/data/geotargets
# Germany: 2276, HH: 1004437, Lund: 1012442, Sweden: 2752
location = '2752'
_DEFAULT_LOCATION_IDS = [location]


# set language
# https://developers.google.com/google-ads/api/reference/data/codes-formats#expandable-7
# German: 1001, English: 1000, Swedish: 1015
language = '1015'
_DEFAULT_LANGUAGE_ID = language


# empty lists for export
report = []
cols = ["keyword", "iteration", "min monthly search", "max monthly search", 
        "mean monthly search", "months", "searches", "annotations", 
        "competition", "competition index", "avg monthly searches (from google)"]


for i in range(1,iterations+1):
    data = []
    
    # skip if current iteration is the first
    if i != 1:
        # get keywords from previous iteration to use as input in this iteration
        file = "%s_%d.xlsx" % (name, i-1)
        prev_i = pd.read_excel(file)
        keywords = prev_i['keyword'].to_list()
        
        # get all previous keywords to check for duplicates in output of this iteration
        check = []
        for prev in range(1,i):
            file = "%s_%d.xlsx" % (name, prev)
            prev = pd.read_excel(file)
            prev_k = prev['keyword'].to_list()
            
            check.extend(prev_k)
        check = list(set(check))

    # check if there are keywords to run
    if len(keywords) != 0:
        
        # keyword planner api only allows 20 queries at a time
        # if more than 20 keywords are used as input,
        # break keywords into chunks of 20 to iterate over
        if len(keywords) > 20:
            
            output = []
            while len(keywords) > 0:
                
                k = keywords[:20]
                batch = func.main(client, customer_id, [location], language, k, None)
                output.extend(batch)
                del keywords[:20]
                time.sleep(6)
        else:
            output = func.main(client, customer_id, [location], language, keywords, None)
            time.sleep(6)


        # iterate over retrieved output to format the data into a table
        for o in output:
            months = []
            searches = []
            annotations = []

            # get search volumes by month
            for v in o.keyword_idea_metrics.monthly_search_volumes:
                months.append(' '.join([v.month.name,str(v.year)]))
                searches.append(v.monthly_searches)

            # get annotations
            for c in o.keyword_annotations.concepts:
                annotations.append(c.concept_group.name)

            # calculate search volume, if empty set to zero
            s = np.array(searches) if len(searches) != 0 else np.array([0])

            # create row adding information to dataframe
            data.append([o.text, i,
                         s.min(), s.max(), s.mean(dtype=int),
                         months, searches, annotations,
                         o.keyword_idea_metrics.competition.name,
                         o.keyword_idea_metrics.competition_index,
                         o.keyword_idea_metrics.avg_monthly_searches])

        # create dataframe from output
        df = pd.DataFrame(data, columns=cols)

        # get number of duplices, both in current output and when compared to previous iterations
        current_duplicate_count = df.duplicated(subset=['keyword'], keep='first').sum()
        if i != 1:
            overall_duplicate_count = df['keyword'].isin(check).sum()
        else:
            overall_duplicate_count = 0

        # remove duplicates
        df.drop_duplicates(subset=['keyword'], keep='first', inplace=True)
        if i != 1:
            df = df[~df['keyword'].isin(check)]
        
        if len(df) != 0:
            # save iteration
            file = "%s_%d.xlsx" % (name, i)
            df.to_excel(file)
            
    
            # save to reporting
            report.append([i, len(output), current_duplicate_count, overall_duplicate_count, df.shape[0]])
        else:
            report.append([i, 'no keywords', current_duplicate_count, overall_duplicate_count, df.shape[0]])
            break
    else:
        report.append([i, 'no keywords', 0, 0])
        break
    
file = "%s_reporting.xlsx" % (name)
reporting = pd.DataFrame(report, columns=["iteration", "total", "duplicates", "duplicates from previous", "final count"])
reporting.to_excel(file)
